# Generar señales

In [87]:
from generate_datasets import make_gravitational_waves
from pathlib import Path

R = 0.65
n_signals = 500
DATA = Path(".")

noisy_signals, gw_signals, labels, Rcoeflist = make_gravitational_waves(
    path_to_data=DATA, n_signals=n_signals, r_min=0.075, r_max=R, n_snr_values=3
)

print(Rcoeflist)
print(f"Number of noisy signals: {len(noisy_signals)}")
print(f"Number of timesteps per series: {len(noisy_signals[0])}")

[0.075  0.3625 0.65  ]
[0.075, 0.36250000000000004, 0.65, 0.075, 0.36250000000000004, 0.65, 0.075, 0.36250000000000004, 0.65, 0.075, 0.36250000000000004, 0.65, 0.075, 0.36250000000000004, 0.65, 0.075, 0.36250000000000004, 0.65, 0.075, 0.36250000000000004, 0.65, 0.075, 0.36250000000000004, 0.65, 0.075, 0.36250000000000004, 0.65, 0.075, 0.36250000000000004, 0.65, 0.075, 0.36250000000000004, 0.65, 0.075, 0.36250000000000004, 0.65, 0.075, 0.36250000000000004, 0.65, 0.075, 0.36250000000000004, 0.65, 0.075, 0.36250000000000004, 0.65, 0.075, 0.36250000000000004, 0.65, 0.075, 0.36250000000000004]
Number of noisy signals: 50
Number of timesteps per series: 8692


# Mapper

In [88]:
import kmapper as km
import sklearn
from sklearn.cluster import AgglomerativeClustering
import numpy as np

Rcoeflist = np.array(Rcoeflist)
gw_signals = np.array(gw_signals)

mapper = km.KeplerMapper(verbose=1)

graph = mapper.map(
    Rcoeflist,
    gw_signals,
    clusterer=sklearn.cluster.DBSCAN(metric='euclidean'),
    cover=km.Cover(n_cubes=6, perc_overlap=0.05),
)

mapper.visualize(
    graph,
    path_html="ondasGravitacionales.html",
    title="Ondas Gravitacionales",
    colorscale=None,
    nbins=8,
    custom_tooltips = Rcoeflist,
    node_color_function=['mean','std','median','max']
    )

km.draw_matplotlib(graph)

KeplerMapper(verbose=1)
Mapping on data shaped (50, 8192) using lens shaped (50,)

Creating 6 hypercubes.

Created 1 edges and 4 nodes in 0:00:00.036998.
Wrote visualization to: ondasGravitacionales.html
no display found. Using non-interactive Agg backend


# Estadístico de prueba

In [89]:
import numpy as np
from sklearn.decomposition import PCA
from gtda.homology import VietorisRipsPersistence
from gtda.time_series import SingleTakensEmbedding
from gtda.diagrams import PersistenceLandscape
import matplotlib.pyplot as plt

In [90]:
from persim.landscapes import PersLandscapeExact
from persim.landscapes import plot_landscape_simple
from persim.landscapes import PersistenceLandscaper
from persim import plot_diagrams

from ripser import ripser


## Función para calcular distancias

### Distancia euclidena

In [91]:
def distancia(diagram):
    suma = 0
    num_diagrams = len(diagram)
    for i in range(min(2, num_diagrams)):  # Verificar el número de diagramas
        h = diagram[i]
        points = h[:-1]
        if len(points) == 0:
            continue
        x_values = np.linspace(0, 2, 10000)
        line_points = np.column_stack((x_values, x_values))
        for point in points:
            if len(point) != 2:
                continue
            distances = np.linalg.norm(line_points - point[:2], axis=1)
            closest_distance = np.min(distances)
            suma += closest_distance
    return suma


### Distancia cuello de botella

In [92]:
# def distancia_cuello_de_botella(diagram):
#     max_distance = 0
#     num_diagrams = len(diagram)
#     for i in range(min(2, num_diagrams)):  # Verificar el número de diagramas
#         h = diagram[i]
#         points = h[:-1]
#         if len(points) == 0:
#             continue
#         x_values = np.linspace(0, 2, 10000)
#         line_points = np.column_stack((x_values, x_values))
#         for point in points:
#             if len(point) != 2:
#                 continue
#             distances = np.linalg.norm(line_points - point[:2], axis=1)
#             farthest_distance = np.max(distances)
#             if farthest_distance > max_distance:
#                 max_distance = farthest_distance
#     return max_distance


# def distancia_cuello_de_botella(diagram):
#     max_distance = 0
#     for point in diagram:
#         distances = np.abs(point[1] - point[0])  # Calcula la distancia a lo largo de la diagonal
#         if distances > max_distance:
#             max_distance = distances
#     return max_distance

### Código Yuu sin modificaciones
Generar diagramas de persistencia para todas las señales

In [93]:
# # Parámetros de incrustación y PCA
# embedding_dimension = 30
# embedding_time_delay = 30
# stride = 5
# pca_components = 3
#  # Inicializar la incrustación y el calculador de homología persistente
# embedder = SingleTakensEmbedding(
#     parameters_type="search", n_jobs=6, time_delay=embedding_time_delay, dimension=embedding_dimension, stride=stride
# )
# persistence = VietorisRipsPersistence(homology_dimensions=[0, 1])

# # Procesar cada señal
# for signal in noisy_signals:
#     # Incrustar la señal
#     signal_embedded = embedder.fit_transform(signal)

#     # Reducir dimensionalidad con PCA
#     signal_embedded_pca = PCA(n_components=pca_components).fit_transform(signal_embedded)

#     # Calcular la homología persistente
#     diagrams = persistence.fit_transform(signal_embedded_pca[None, :, :])

# Parámetros de incrustación y PCA
embedding_dimension = 30
embedding_time_delay = 30
stride = 5
pca_components = 3

# Inicializar la incrustación y el calculador de homología persistente
embedder = SingleTakensEmbedding(
    parameters_type="search", n_jobs=6, time_delay=embedding_time_delay, dimension=embedding_dimension, stride=stride
)
persistence = VietorisRipsPersistence(homology_dimensions=[0, 1])

# Variables para almacenar diagramas y distancias
all_diagrams = []
all_distances = []

# Procesar cada señal
for signal in noisy_signals:
    # Incrustar la señal
    signal_embedded = embedder.fit_transform(signal)

    # Reducir dimensionalidad con PCA
    signal_embedded_pca = PCA(n_components=pca_components).fit_transform(signal_embedded)

    # Calcular la homología persistente
    diagrams = persistence.fit_transform(signal_embedded_pca[None, :, :])
    
    # Guardar los diagramas en la lista
    all_diagrams.append(diagrams)

    # Calcular la distancia y guardarla en la lista
    distance = distancia(diagrams)
    all_distances.append(distance)

In [97]:
# # Con for que itera sobre todas las señales
# def slidding_window(señal):
#     embedding_dimension = 30
#     embedding_time_delay = 30
#     stride = 5
#     pca_components = 3

#     # Inicializar la incrustación y el calculador de homología persistente
#     embedder = SingleTakensEmbedding(
#         parameters_type="search", n_jobs=6, time_delay=embedding_time_delay, dimension=embedding_dimension, stride=stride
#     )
#     persistence = VietorisRipsPersistence(homology_dimensions=[0, 1])

#     # Procesar cada señal
#     for signal in noisy_signals:
#         # Incrustar la señal
#         signal_embedded = embedder.fit_transform(signal)

#         # Reducir dimensionalidad con PCA
#         signal_embedded_pca = PCA(n_components=pca_components).fit_transform(signal_embedded)

#     return signal_embedded_pca


def slidding_window(senal):
    embedding_dimension = 30
    embedding_time_delay = 30
    stride = 5
    pca_components = 3

    # Inicializar la incrustación y el calculador de homología persistente
    embedder = SingleTakensEmbedding(
        parameters_type="search", n_jobs=6, time_delay=embedding_time_delay, dimension=embedding_dimension, stride=stride
    )
    persistence = VietorisRipsPersistence(homology_dimensions=[0, 1])

    # Incrustar la señal
    signal_embedded = embedder.fit_transform(senal)

    # Reducir dimensionalidad con PCA
    signal_embedded_pca = PCA(n_components=pca_components).fit_transform(signal_embedded)

    return signal_embedded_pca



## Grupo Rcoef 0.075 y 0.65

In [101]:
def estadistico_global(grupo1, grupo2):
    # Calculo la primera parte de la función de costo
    nm1 = len(grupo1)
    nm2 = len(grupo2)
    coeficiente1 = 1 / (2 * nm1 * (nm1 - 1))
    coeficiente2 = 1 / (2 * nm2 * (nm2 - 1))

    # Cálculo de los diagramas de persistencia para grupo1
    diagramas_presistencia_grupo1 = []
    for senal_grupo_1 in grupo1:
        y_noise_embedded_pca_1 = slidding_window(senal_grupo_1)
        diagrams_grupo1 = ripser(y_noise_embedded_pca_1)['dgms']
        diagramas_presistencia_grupo1.append(diagrams_grupo1)
    
    # Cálculo de los diagramas de persistencia para grupo2
    diagramas_persistencia_grupo_2 = []
    for senal_grupo_2 in grupo2:
        y_noise_embedded_pca_2 = slidding_window(senal_grupo_2)
        diagrams_grupo2 = ripser(y_noise_embedded_pca_2)['dgms']
        diagramas_persistencia_grupo_2.append(diagrams_grupo2)

    # Cálculo y suma de la función de distancia para el grupo 1
    suma_distancias_grupo1 = 0
    suma_distancias_grupo2 = 0
    for diagrama1 in diagramas_presistencia_grupo1:
        suma_senal_1 = distancia(diagrama1)
        suma_distancias_grupo1 += suma_senal_1
  
    for diagrama2 in diagramas_persistencia_grupo_2:
        suma_senal_2 = distancia(diagrama2)
        suma_distancias_grupo2 += suma_senal_2
    
    estadistico = (coeficiente1 * suma_distancias_grupo1) + (coeficiente2 * suma_distancias_grupo2)
    return diagramas_presistencia_grupo1, diagramas_persistencia_grupo_2, estadistico



In [102]:
def estadistico_subconjunto(diagramas_grupo1, diagramas_grupo2):

    nm1, nm2 = len(diagramas_grupo1), len(diagramas_grupo2)
    coeficiente1, coeficiente2 = 1 / (2 * nm1 * (nm1 - 1)), 1 / (2 * nm2 * (nm2 - 1))

    suma_distancias_grupo1 = sum(distancia(diagrama) for diagrama in diagramas_grupo1)
    suma_distancias_grupo2 = sum(distancia(diagrama) for diagrama in diagramas_grupo2)
    
    estadistico = (coeficiente1 * suma_distancias_grupo1) + (coeficiente2 * suma_distancias_grupo2)
    return estadistico

In [103]:
# Convertir la lista Rcoeflist a un array numpy para facilitar las operaciones
Rcoefarray = np.array(Rcoeflist)

# Obtener los índices para cada grupo
indices_075 = np.where(Rcoefarray == 0.075)[0]
indices_3625 = np.where(Rcoefarray == 0.36250000000000004)[0]
indices_65 = np.where(Rcoefarray == 0.65)[0]

# Crear listas con las señales de cada grupo
senales_075 = [noisy_signals[index] for index in indices_075]
senales_3625 = [noisy_signals[index] for index in indices_3625]
senales_65 = [noisy_signals[index] for index in indices_65]


## Grupo Rcoef 0.075 y 0.65

In [104]:
diagrama_persistencia_075,diagrama_persistencia_3625,estadistico=estadistico_global(senales_075,senales_3625)
print("Estadistico Global entre grupos de R Coef 0.075 y 0.3625= ",estadistico)

Estadistico Global entre grupos de R Coef 0.075 y 0.3625=  4.830643208229783e-17


In [105]:
import random
def permutacion_Grupo075_Grupo3625(grupo1,grupo2,num_permutaciones):
    z=1
    for i in range(num_permutaciones):
        random_number1 = random.randint(5, 10)
        random_number2 = random.randint(5, 10)#genero un numero random entre 5 y 20 para crear los subconuntos que siguen la proporcion de la poblacion
        subconjunto_promedio_grupo_1 = random.sample(grupo1, random_number1)
        subconjunto_promedio_grupo_2 = random.sample(grupo2, random_number2)
        estadistico=estadistico_subconjunto(subconjunto_promedio_grupo_1,subconjunto_promedio_grupo_2)

        if estadistico<=4.830643208229783e-17:
            z+=1
    
    den=num_permutaciones+1
    pval = z/den
    if pval<= 0.05:
        print("Rechazo H0. Hay diferencias significativas en la topología de las señales entre diferentes niveles de coeficientes de escala de ruido.")
    else:
        print("No Rechazo H0. No hay diferencias significativas en la topología de las señales entre diferentes niveles de coeficientes de escala de ruido")
    return pval

In [106]:

pval=permutacion_Grupo075_Grupo3625(diagrama_persistencia_075,diagrama_persistencia_3625,25)
print("Obtuvimos un pvalor de ",pval)

Rechazo H0. No hay diferencias significativas en la topología de las señales entre diferentes niveles de coeficientes de escala de ruido.
Obtuvimos un pvalor de  0.038461538461538464


## Grupo Rcoef 0.3625 y 0.65

In [113]:
diagrama_persistencia_3625,diagrama_persistencia_65,estadistico_2=estadistico_global(senales_3625,senales_65)
print("Estadistico Global entre grupos de R Coef 0.3625 y 0.65= ",estadistico_2)

Estadistico Global entre grupos de R Coef 0.3625 y 0.65=  1.3504457806808525e-17


In [116]:
def permutacion_Grupo3625_Grupo65(grupo1,grupo2,num_permutaciones):
    z=1
    for i in range(num_permutaciones):
        random_number1 = random.randint(5, 10)
        random_number2 = random.randint(5, 10)#genero un numero random entre 5 y 20 para crear los subconuntos que siguen la proporcion de la poblacion
        subconjunto_promedio_grupo_1 = random.sample(grupo1, random_number1)
        subconjunto_promedio_grupo_2 = random.sample(grupo2, random_number2)
        estadistico=estadistico_subconjunto(subconjunto_promedio_grupo_1,subconjunto_promedio_grupo_2)
        if estadistico<=1.3504457806808525e-17:
            z+=1
    
    den=num_permutaciones+1
    pval = z/den
    if pval<= 0.05:
        print("Rechazo H0. Hay diferencias significativas en la topología de las señales entre diferentes niveles de coeficientes de escala de ruido.")
    else:
        print("No Rechazo H0. No hay diferencias significativas en la topología de las señales entre diferentes niveles de coeficientes de escala de ruido")
    return pval

In [118]:
pval3=permutacion_Grupo3625_Grupo65(diagrama_persistencia_3625,diagrama_persistencia_65,25)
print("Obtuvimos un pvalor de ",pval3)

Rechazo H0. Hay diferencias significativas en la topología de las señales entre diferentes niveles de coeficientes de escala de ruido.
Obtuvimos un pvalor de  0.038461538461538464


## Grupo Rcoef 0.75 y 0.65

In [110]:
diagrama_persistencia_075,diagrama_persistencia_65,estadistico_2=estadistico_global(senales_075,senales_65)
print("Estadistico Global entre grupos de R Coef 0.075 y 0.65= ",estadistico_2)

Estadistico Global entre grupos de R Coef 0.075 y 0.65=  4.497106528549091e-17


In [111]:
def permutacion_Grupo075_Grupo65(grupo1,grupo2,num_permutaciones):
    z=1
    for i in range(num_permutaciones):
        random_number1 = random.randint(5, 10)
        random_number2 = random.randint(5, 10)#genero un numero random entre 5 y 20 para crear los subconuntos que siguen la proporcion de la poblacion
        subconjunto_promedio_grupo_1 = random.sample(grupo1, random_number1)
        subconjunto_promedio_grupo_2 = random.sample(grupo2, random_number2)
        estadistico=estadistico_subconjunto(subconjunto_promedio_grupo_1,subconjunto_promedio_grupo_2)

        if estadistico<=4.5217561264029805e-17:
            z+=1
    
    den=num_permutaciones+1
    pval = z/den
    if pval<= 0.05:
        print("Rechazo H0. Hay diferencias significativas en la topología de las señales entre diferentes niveles de coeficientes de escala de ruido.")
    else:
        print("No Rechazo H0. No hay diferencias significativas en la topología de las señales entre diferentes niveles de coeficientes de escala de ruido")
    return pval

In [112]:
pval2=permutacion_Grupo075_Grupo65(diagrama_persistencia_075,diagrama_persistencia_65,25)
print("Obtuvimos un pvalor de ",pval2)

Rechazo H0. No hay diferencias significativas en la topología de las señales entre diferentes niveles de coeficientes de escala de ruido.
Obtuvimos un pvalor de  0.038461538461538464
